In [ ]:
!pip install google-colab
!pip install pandas

# Authenticate

In [ ]:
from google.colab import auth
auth.authenticate_user()
import gspread
from oauth2client.client import GoogleCredentials
gc = gspread.authorize(GoogleCredentials.get_application_default())

# File import

## Drive

In [ ]:
!pip install PyDrive

In [ ]:
import pandas as pd


In [ ]:
#@Sheet url
drive_url = "" #@param {type:"string"}
wb = gc.open_by_url(drive_url)

In [ ]:
sheet_name = "bad_emails_INBOX.Archives.2020-09" #@param {type:"string"}
sheet = wb.worksheet(sheet_name)


## To pandas 

In [ ]:
data = sheet.get_all_values()
df = pd.DataFrame(data)

In [ ]:
df.head()

In [ ]:
new_header = df.iloc[0] #grab the first row for the header
df = df[1:] #take the data less the header row
df.columns = new_header #set the header row as the df header

## Get "Date" to date

In [ ]:
df.info() 


In [ ]:
df["Date"].replace({"None": "1970-01-01T00:00:00"})


In [ ]:
df.loc[df['Date'].isnull(), 'Date'] = '1970-01-01T00:00:00'

In [ ]:
pd.set_option('display.max_rows', df.shape[0]+1)


In [ ]:
df['Date']

In [ ]:
# convert the 'Date' column to datetime format 

df['Date']= pd.to_datetime(df['Date'], utc=True) 


In [ ]:
df.info()

In [ ]:
# change format
df['Date'] = pd.to_datetime(df["Date"].dt.strftime('%Y-%m-%dT%H:%M:%S'))


In [ ]:

df = df.rename(columns={"Date": "datetime"}) # rename to better timesketch import


In [ ]:
df.head()

In [ ]:
# Timesketch

## Setup

In [ ]:
!pip install timesketch_api_client 
!pip install timesketch_import_client

from timesketch_api_client import client
from timesketch_import_client import importer

In [ ]:
SERVER_LOCATION = "http://localhost:5000"
USERNAME = "dev"
PASSWORD = "dev"

ts = client.TimesketchApi(SERVER_LOCATION, USERNAME, PASSWORD)
my_sketch = ts.get_sketch(1)

## Upload

In [ ]:
with importer.ImportStreamer() as streamer:
    streamer.set_sketch(my_sketch)
    streamer.set_timestamp_description('malicious attachments')
    streamer.set_timeline_name('malicious attachments')
    streamer.set_message_format_string(
        'Malicious attachment entry {Subject:s} {Attachment:s} {Malware:s}')

    streamer.add_data_frame(df)